In [1]:
import pandas as pd
import pylab as plt 
import seaborn as sns
import numpy as np
import math
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
Labor_Market_ds = pd.read_csv('Labor_Market_Data_Cleaning.csv')
Labor_Market_ds.head(3)

,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,YEAR,TIPO,NAZIONE
0,F,49,Rifugi di montagna,LICENZA MEDIA,LAVORO INTERMITTENTE,NON DEFINITO,BERGAMO,2021,ATTIVATO,ITALIA
1,M,16,"Costruzione di strade, autostrade e piste aero...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,2021,ATTIVATO,ITALIA
2,M,19,Ristorazione con somministrazione,LICENZA MEDIA,LAVORO INTERMITTENTE,NON DEFINITO,BERGAMO,2021,ATTIVATO,ITALIA


In [3]:
#Delete the column "YEAR" from the dataset
Labor_Market_ds_new =Labor_Market_ds.drop(['YEAR'], axis=1)

## Data Transformation
### Provinces Transformation

In [4]:
#Extraction of data relating to the provinces of Sondrio, Milan and Varese 
dfinale=Labor_Market_ds_new.apply(lambda row: row[Labor_Market_ds_new['PROVINCIAIMPRESA'].isin(['SONDRIO','VARESE','MILANO'])])
dfinale.PROVINCIAIMPRESA.unique()

array(['MILANO', 'VARESE', 'SONDRIO'], dtype=object)

In [5]:
#Obtained data are mapped to associate different values to each category
scale_mapper = {'MILANO': 0, 
                'VARESE': 1,
                'SONDRIO': 2,
               }

In [6]:
#New column containing related integer values is created 
dfinale['SCALEPROVINCIA'] = dfinale['PROVINCIAIMPRESA'].replace(scale_mapper)

### Provinces Transformation

In [7]:
#Sorting of data relating to workers of the 5 most populous nations 
aa = dfinale.groupby('NAZIONE').NAZIONE.count().sort_values(ascending=False).head().index.tolist()
aa

['ITALIA', 'EGITTO', 'ROMANIA', 'ALBANIA', 'MAROCCO']

In [8]:
#Extraction of data relating to workers of the 5 most populous nations 
dfinale= dfinale.apply(lambda row: row[Labor_Market_ds_new['NAZIONE'].isin(aa)])

In [9]:
#Obtained data are mapped to associate different values to each category
scale_mapper = {'ITALIA': 0, 
                'EGITTO': 1,
                'ROMANIA': 2,
                'ALBANIA': 3,
                'MAROCCO': 4, 
               }

In [10]:
#New column containing related integer values is created
dfinale['SCALENAZIONE'] = dfinale['NAZIONE'].replace(scale_mapper)
dfinale.SCALENAZIONE.unique()

array([0, 2, 1, 4, 3], dtype=int64)

In [11]:
dfinale.reset_index(inplace=True, drop=True)

### Contract Type Transformation

In [12]:
#Searching different type of the Contract
dfinale.CONTRATTO.unique()

array(['TIROCINIO', 'LAVORO A TEMPO DETERMINATO',
       'LAVORO A TEMPO INDETERMINATO', 'LAVORO INTERMITTENTE',
       'LAVORO DOMESTICO', 'LAVORO A TEMPO DETERMINATO  PER SOSTITUZIONE',
       'APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO DI MESTIERE',
       'COLLABORAZIONE COORDINATA E CONTINUATIVA',
       'CONTRATTI DI BORSA LAVORO E ALTRE WORK EXPERIENCES',
       'APPRENDISTATO PER LA QUALIFICA E PER IL DIPLOMA PROFESSIONALE, IL DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE E IL CERTIFICATO DI SPECIALIZZAZIONE TECNICA SUPERIORE',
       'LAVORO AUTONOMO NELLO SPETTACOLO',
       'LAVORO O ATTIVITÀ SOCIALMENTE UTILE (LSU - ASU)',
       'LAVORO A DOMICILIO',
       'Lavoro a tempo indeterminato con piattaforma',
       'CONTRATTO DI FORMAZIONE LAVORO (SOLO PUBBLICA AMMINISTRAZIONE)',
       'CONTRATTO DI AGENZIA',
       'Lavoro a tempo determinato per sostituzione con piattaforma',
       'APPRENDISTATO DI ALTA FORMAZIONE E RICERCA',
       'Lavoro a tempo determinato con piattaf

In [13]:
#Obtained data are mapped to associate different values to each category
scale_mapper = {'LAVORO A TEMPO DETERMINATO': 0, 
                'LAVORO A TEMPO DETERMINATO  PER SOSTITUZIONE': 0,
                'Lavoro a tempo determinato con piattaforma': 0,
                'Lavoro a tempo determinato per sostituzione con piattaforma': 0,
                'LAVORO A TEMPO INDETERMINATO': 1,
                'Lavoro a tempo indeterminato con piattaforma': 1,
                'APPRENDISTATO DI ALTA FORMAZIONE E RICERCA': 2,
                'APPRENDISTATO PER LA QUALIFICA E PER IL DIPLOMA PROFESSIONALE, IL DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE E IL CERTIFICATO DI SPECIALIZZAZIONE TECNICA SUPERIORE': 2,
                'APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO DI MESTIERE': 2,
                'TIROCINIO': 3,
                'LAVORO INTERMITTENTE': 3,
                'LAVORO DOMESTICO': 3,
                'COLLABORAZIONE COORDINATA E CONTINUATIVA': 3,
                'CONTRATTI DI BORSA LAVORO E ALTRE WORK EXPERIENCES': 3,
                'LAVORO AUTONOMO NELLO SPETTACOLO': 3,
                'LAVORO O ATTIVITÀ SOCIALMENTE UTILE (LSU - ASU)': 3,
                'LAVORO A DOMICILIO': 3,
                'CONTRATTO DI FORMAZIONE LAVORO (SOLO PUBBLICA AMMINISTRAZIONE)': 3,
                'CONTRATTO DI AGENZIA': 3,
                'LAVORO CONGIUNTO IN AGRICOLTURA': 3,
                'LAVORO RIPARTITO': 3,
                'ASSOCIAZIONE IN PARTECIPAZIONE': 3,
               }

In [14]:
#New column containing related integer values is created
dfinale['SCALECONTRATTO'] = dfinale['CONTRATTO'].replace(scale_mapper)
dfinale.SCALECONTRATTO.unique()

array([3, 0, 1, 2], dtype=int64)

### Economic Sector Transformation

In [15]:
#Searching different type of the Economic Sector
dfinale.groupby('SETTOREECONOMICODETTAGLIO').SETTOREECONOMICODETTAGLIO.count().sort_values(ascending=False).head()

SETTOREECONOMICODETTAGLIO
Attività di produzione cinematografica, di video e di programmi televisivi    33247
Ristorazione con somministrazione                                             22369
Pulizia generale (non specializzata) di edifici                               21026
Costruzione di edifici residenziali e non residenziali                        18776
Servizi logistici relativi alla distribuzione delle merci                     18262
Name: SETTOREECONOMICODETTAGLIO, dtype: int64

In [16]:
#Obtained data are mapped to associate different values to each category
scale_mapper = {'Attività di produzione cinematografica, di video e di programmi televisivi': 0, 
                'Ristorazione con somministrazione': 1,
                'Pulizia generale (non specializzata) di edifici': 2,
                'Costruzione di edifici residenziali e non residenziali': 3,
                'Servizi logistici relativi alla distribuzione delle merci': 4, 
               }

In [17]:
#New column containing related integer values is created
dfinale['SCALESETTORE'] = dfinale['SETTOREECONOMICODETTAGLIO'].replace(scale_mapper)
dfinale = dfinale.apply(lambda row: row[dfinale['SCALESETTORE'].isin([0,1,2,3,4])])
dfinale.SCALESETTORE.unique()

array([3, 2, 1, 4, 0], dtype=object)

### Labor Type Transformation

In [18]:
#Searching different type of the Labor Type
dfinale.groupby('TIPO').TIPO.count()

TIPO
ATTIVATO       65276
CESSATO        14191
PROROGATO      20966
TRASFORMATO    13247
Name: TIPO, dtype: int64

In [19]:
#Obtained data are mapped to associate different values to each category
scale_mapper = {'ATTIVATO': 0, 
                'CESSATO': 1,
                'PROROGATO': 2,
                'TRASFORMATO': 3,
               }

In [20]:
#New column containing related integer values is created
dfinale['SCALETIPO'] = dfinale['TIPO'].replace(scale_mapper)
dfinale.SCALETIPO.unique()

array([0, 1, 2, 3], dtype=int64)

### Labor Modality Transformation

In [21]:
#Searching different type of the Labor Type
dfinale.groupby('MODALITALAVORO').MODALITALAVORO.count()

MODALITALAVORO
NON DEFINITO                  10418
TEMPO PARZIALE MISTO           5866
TEMPO PARZIALE ORIZZONTALE    28765
TEMPO PARZIALE VERTICALE       1778
TEMPO PIENO                   66853
Name: MODALITALAVORO, dtype: int64

In [22]:
#Obtained data are mapped to associate different values to each category
scale_mapper = {'NON DEFINITO': 0, 
                'TEMPO PARZIALE MISTO': 1,
                'TEMPO PARZIALE ORIZZONTALE': 2,
                'TEMPO PARZIALE VERTICALE': 3,
                'TEMPO PIENO': 4,
               }

In [23]:
#New column containing related integer values is created
dfinale['SCALEMOD'] = dfinale['MODALITALAVORO'].replace(scale_mapper)

### Gender Transformation

In [24]:
#Obtained data are mapped to associate different values to each category
scale_mapper = {'F': 0, 
                'M': 1,
               }

In [25]:
#New column containing related integer values is created
dfinale['SCALEGENERE'] = dfinale['GENERE'].replace(scale_mapper)

In [26]:
dfinale.reset_index(inplace=True, drop=True)

### Age Transformation and Study Title Transfomation 

In [27]:
dfinale['SCALETA'] = 0
dfinale['SCALETITOLO'] = 0
for i in range(dfinale.ETA.size):
    if dfinale['ETA'].loc[i] < 40:  dfinale['SCALETA'].loc[i]= 0
    elif dfinale['ETA'].loc[i]>= 40 and dfinale['ETA'].loc[i]< 55: dfinale['SCALETA'].loc[i]= 1
    else : dfinale['SCALETA'].loc[i]= 2

    if dfinale['TITOLOSTUDIO'].loc[i] == 'NESSUN TITOLO DI STUDIO':  dfinale['SCALETITOLO'].loc[i]= 0
    elif dfinale['TITOLOSTUDIO'].loc[i] == 'LICENZA ELEMENTARE' : dfinale['SCALETITOLO'].loc[i]= 1
    elif dfinale['TITOLOSTUDIO'].loc[i] == 'LICENZA MEDIA' :  dfinale['SCALETITOLO'].loc[i]= 2  
    elif dfinale['TITOLOSTUDIO'].loc[i] == "DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE PERMETTE L'ACCESSO ALL'UNIVERSITA" or dfinale['TITOLOSTUDIO'].loc[i] == "TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOLASTICA ED EXTRA-SCOLASTICA) CHE NON PERMETTE L'ACCESSO ALL'UNIVERSITÀ ()":  dfinale['SCALETITOLO'].loc[i]= 3
    else : dfinale['SCALETITOLO'].loc[i]= 4        

C:\Users\daddc\anaconda3\envs\Ambiente\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [28]:
#Creation of the final Prediction Matrix
preparedf= dfinale[['SCALEGENERE','SCALETA','SCALESETTORE','SCALETITOLO','SCALECONTRATTO','SCALEMOD','SCALEPROVINCIA','SCALETIPO','SCALENAZIONE']]

In [29]:
preparedf

,SCALEGENERE,SCALETA,SCALESETTORE,SCALETITOLO,SCALECONTRATTO,SCALEMOD,SCALEPROVINCIA,SCALETIPO,SCALENAZIONE
0,1,1,3,0,0,4,0,0,2
1,1,0,3,0,0,4,0,0,1
2,1,0,3,0,1,2,0,0,1
3,0,1,2,2,0,4,0,0,0
4,1,0,1,0,3,0,0,0,2
...,...,...,...,...,...,...,...,...,...
113675,1,0,4,0,0,4,0,3,2
113676,1,1,1,0,0,2,0,3,0
113677,0,0,1,3,0,4,0,3,0
113678,0,0,4,0,0,4,0,3,2


In [30]:
#Creation of the final Prediction Matrix DataSet
preparedf.to_csv('Labor_Market_Data_Transforming.csv', index = False, header = True)